In [1]:
# Importing libraries
import pandas as pd                   # data science essentials
import matplotlib.pyplot as plt       # essential graphical output
import seaborn as sns                 # enhanced graphical output
import numpy as np                    # mathematical essentials
import statsmodels.formula.api as smf # regression modeling
from sklearn.model_selection import train_test_split# train/test split
import sklearn.linear_model # linear models

# Step 1
# Setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# Specifying file name
file = './__datasets/Assignment/birthweight_low.xlsx'

# Reading the file into Python
birthweight = pd.read_excel(file)

# Check features and Observations
print(f"""
Size of original data set:
--------------------------
Observations: {birthweight.shape[0]}
Features:     {birthweight.shape[1]}
""")

# Check for NAN in features
print(f"""
Features with missing values: 
-----------------------------
{birthweight.isna().any()}

""")

#Keep Original data set intact (This could be referred to at a later stage for data integrity check)
birthweight_Original = birthweight.copy()

# Flag missing values
birthweight["m_meduc"] = birthweight["meduc"]
birthweight["m_npvis"] = birthweight["npvis"]
birthweight["m_feduc"] = birthweight["feduc"]

# Impute missing values with the mean. For all variables mean appears to be a safe technique given 
# that they could come from the same location. In the absence of the socio-economic data we will go with the mean.

birthweight["m_meduc"].fillna(round(birthweight["meduc"].mean(), 1), inplace=True)
birthweight["m_npvis"].fillna(round(birthweight["npvis"].mean(), 1), inplace=True)
birthweight["m_feduc"].fillna(round(birthweight["feduc"].mean(), 1), inplace=True)

# Drop features with missing values from the orginal data set, to avoid accidently including them as predictors
birthweight = birthweight.drop(["meduc","npvis","feduc"], axis = 1)

# Alert that there has been a statistical violation, should it be the case
if birthweight.shape[0] < 30:
    
    print("""NB: You are attempting to build a model with an inadequate sample size. \nTo achieve a normal distribution in sample size it is recommended to have at least 30 observations.""")

#Features with missing values after imputation of the data
print(f"""
Missing values after imputation: 
--------------------------------
{birthweight.isna().any()}

""")

#Display the head to have a glance at the data after imputation
#print(birthweight.tail(n=5))



Size of original data set:
--------------------------
Observations: 196
Features:     18


Features with missing values: 
-----------------------------
mage      False
meduc      True
monpre    False
npvis      True
fage      False
feduc      True
omaps     False
fmaps     False
cigs      False
drink     False
male      False
mwhte     False
mblck     False
moth      False
fwhte     False
fblck     False
foth      False
bwght     False
dtype: bool



Missing values after imputation: 
--------------------------------
mage       False
monpre     False
fage       False
omaps      False
fmaps      False
cigs       False
drink      False
male       False
mwhte      False
mblck      False
moth       False
fwhte      False
fblck      False
foth       False
bwght      False
m_meduc    False
m_npvis    False
m_feduc    False
dtype: bool




In [2]:
# Check for null values summing together the results per column
print(f"""
Result of null value(s) per column
----------------------------------
{birthweight.isnull().sum(axis = 0)}

""")



Result of null value(s) per column
----------------------------------
mage       0
monpre     0
fage       0
omaps      0
fmaps      0
cigs       0
drink      0
male       0
mwhte      0
mblck      0
moth       0
fwhte      0
fblck      0
foth       0
bwght      0
m_meduc    0
m_npvis    0
m_feduc    0
dtype: int64




In [3]:
# Create a list with all the columns to be used for the linear correlation
continuous_data = ["bwght", "fage", "monpre", "m_npvis", "mage",
                   "omaps", "fmaps", "cigs", "drink", "male", "mwhte", "mblck", "fwhte", "fblck", "m_feduc", "m_meduc"]

# Develop a correlation matrix based on continuous features
birthweight_corr = birthweight[continuous_data].corr(method = "pearson")

# Smoking and Drinking have an effect on the baby's weight, let's interact them
birthweight["drink_smoke"] = birthweight["drink"] * birthweight["cigs"]  

# lets develop a correlation matrix against baby weight
birthweight_corr = birthweight.loc[ : , ["drink",
                              "cigs",
                              "drink_smoke",
                              "bwght"]  ].corr(method = "pearson")\
                                             .round(decimals = 2)

#Remove the features after interaction
birthweight = birthweight.drop(["cigs", "drink"], axis = 1)


# Print the correlation to assess the impact of the transformation done above
print(f"""
Smoking and Drinking correlation to baby weight
===============================================

{birthweight_corr['bwght']}

""")




Smoking and Drinking correlation to baby weight

drink         -0.74
cigs          -0.57
drink_smoke   -0.80
bwght          1.00
Name: bwght, dtype: float64




In [4]:
# Step 2

# Prepare response variables
birthweight_target = birthweight.loc[ : , "bwght"]

# preparing explanatory variable data
birthweight_data   = birthweight.drop(["bwght"], axis = 1)

# Check the features and Observations
print(f"""
Size of Dataset after feature engineering
------------------------------------------
Observations: {birthweight.shape[0]}
Features:     {birthweight.shape[1]}
""")

# Preparing training and testing data sets
x_train, x_test, y_train, y_test = train_test_split(
            birthweight_data,
            birthweight_target,
            test_size = 0.25,
            random_state = 219)


# checking the shapes of the data sets
print(f"""
Training Data
-------------
X-side: {x_train.shape}
y-side: {y_train.shape}


Testing Data
------------
X-side: {x_test.shape}
y-side: {y_test.shape}
""")

#Remove the response variable
birthweight = birthweight.drop(["bwght"], axis = 1)

# Check the features we are going to train
print(f"""
Features of the data set
-------------------------
Features: {birthweight.columns}
""")



Size of Dataset after feature engineering
------------------------------------------
Observations: 196
Features:     17


Training Data
-------------
X-side: (147, 16)
y-side: (147,)


Testing Data
------------
X-side: (49, 16)
y-side: (49,)


Features of the data set
-------------------------
Features: Index(['mage', 'monpre', 'fage', 'omaps', 'fmaps', 'male', 'mwhte', 'mblck', 'moth', 'fwhte', 'fblck', 'foth', 'm_meduc', 'm_npvis', 'm_feduc', 'drink_smoke'], dtype='object')



In [456]:

# merging X_train and y_train so that they can be used in statsmodels
birthweight_train = pd.concat([x_train, y_train], axis = 1)

# Setup the model. This include removed features that have shot past an acceptable p-Value and features that cannot be controlled
lm_best = smf.ols(formula =  """bwght ~  mage +
                                                mwhte +
                                                mblck +
                                                male +
                                                moth +
                                                m_meduc +
                                                drink_smoke
                                                 """,
                                data = birthweight_train)

# Fit the model based on the data
results = lm_best.fit()

# Analyze the summary output
print(results.summary())



                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.708
Method:                 Least Squares   F-statistic:                     60.04
Date:                Wed, 24 Nov 2021   Prob (F-statistic):           2.60e-36
Time:                        21:39:04   Log-Likelihood:                -1071.1
No. Observations:                 147   AIC:                             2156.
Df Residuals:                     140   BIC:                             2177.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    2909.8873    203.595     14.293      

In [5]:

# INSTANTIATING a model object
ard_model = sklearn.linear_model.ARDRegression()


# FITTING the training data
ard_fit = ard_model.fit(x_train, y_train)


# PREDICTING on new data
ard_pred = ard_fit.predict(x_test)


print('ARD Training Score (FINAL):', ard_model.score(x_train, y_train))
print('ARD Testing Score :',  ard_model.score(x_test, y_test))


# saving scoring data for future use
ard_train_score =  ard_model.score(x_train, y_train).round(4)
ard_test_score  = ard_model.score(x_test, y_test).round(4)


# displaying and saving the gap between training and testing
print('ARD Train-Test Gap :', (abs(ard_train_score - ard_test_score)).round(4))
ard_test_gap = (abs(ard_train_score - ard_test_score)).round(4)


ARD Training Score (FINAL): 0.7238625269210536
ARD Testing Score : 0.5371802724461493
ARD Train-Test Gap : 0.1867


In [6]:

# INSTANTIATING a model object
lasso_model = sklearn.linear_model.Lasso(alpha = 1.0, normalize = True) # default magitude


# FITTING to the training data
lasso_fit = lasso_model.fit(x_train, y_train)


# PREDICTING on new data
lasso_pred = lasso_fit.predict(x_test)


# SCORING the results
print('Lasso Training Score :', lasso_model.score(x_train, y_train).round(4))
print('Lasso Testing Score  :', lasso_model.score(x_test, y_test).round(4))


## the following code has been provided for you ##

# saving scoring data for future use
lasso_train_score = lasso_model.score(x_train, y_train).round(4) # using R-square
lasso_test_score  = lasso_model.score(x_test, y_test).round(4)   # using R-square


# displaying and saving the gap between training and testing
print('Lasso Train-Test Gap :', abs(lasso_train_score - lasso_test_score).round(4))
lasso_test_gap = abs(lasso_train_score - lasso_test_score).round(4)

Lasso Training Score : 0.7268
Lasso Testing Score  : 0.5258
Lasso Train-Test Gap : 0.201
